In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


# Build Certified Iris Region Object

In [6]:
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}

In [7]:
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)

In [8]:
seed_points = np.tan(np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])    
                        /2)

regions, ellipses = iris_generator.iris_in_rational_space(seed_points)


0
snopt_example=[ 0.35529068 -0.22984731  0.02559963], growth = 0.05990553085918943
snopt_example=[ 0.28673366 -0.30128467  0.33402563], growth = 0.09485392272232375
snopt_example=[ 0.28776043 -0.30045108  0.32725976], growth = 0.09339195802962706
terminating because a required containment point would have not been contained
Time:   1.60 	Volume:  21.77 	Center: [1.44312073e-09 1.44280166e-09 1.44440504e-09]
0
snopt_example=[ 0.35529266 -0.22984424  0.02559962], growth = 0.05990553077553568
snopt_example=[ 0.30804713 -0.27858144  0.24207296], growth = 0.07703332293708175
terminating because a required containment point would have not been contained
Time:   1.02 	Volume:  21.77 	Center: [1.44312073e-09 1.44280166e-09 1.44440504e-09]
snopt_example=[ 0.30547805 -0.52068241  0.30020753], growth = 354169277.01944494
snopt_example=[ 0.19754042 -0.45436384  0.26846797], growth = 33224401.49490118
snopt_example=[ 0.19598983 -0.45413506  0.27106354], growth = 32414169.41665411
0
snopt_example=[

In [9]:
trajectory_start = np.tan(np.array([-1, -2, 0.5])/2) # seed_points[1,:]
trajectory_end = np.tan(np.array([0.25, -1.6, -0.25])/2)  #seed_points[3,:]

In [ ]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions, ellipses)
    visualizer.plot_seedpoints(seed_points)
    visualizer.visualize_collision_constraint(N = 70)
    

## Certify the regions

In [11]:
iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

Time to initialize region program: 1.1363
time to create Region Certification Problem: : 0.6416
Time to initialize region program: 1.1046
time to create Region Certification Problem: : 0.6314
Time to initialize region program: 1.3222
time to create Region Certification Problem: : 1.6334
Time to initialize region program: 1.5366
time to create Region Certification Problem: : 2.8068
Time to initialize region program: 1.2339
time to create Region Certification Problem: : 1.1955
Time to initialize region program: 1.2856
time to create Region Certification Problem: : 1.0104


{<pydrake.geometry.optimization.HPolyhedron at 0x7fa0ed7ad4f0>: <certified_iris_generator.RegionCertificationProblem at 0x7fa12c717fa0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fa12c7088f0>: <certified_iris_generator.RegionCertificationProblem at 0x7fa0ed77dd90>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fa0f5ef3fb0>: <certified_iris_generator.RegionCertificationProblem at 0x7fa0ed78f280>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fa0ed7b6eb0>: <certified_iris_generator.RegionCertificationProblem at 0x7fa12c7179a0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fa0ed7b6f70>: <certified_iris_generator.RegionCertificationProblem at 0x7fa0ed77dfa0>,
 <pydrake.geometry.optimization.HPolyhedron at 0x7fa0ed7b6430>: <certified_iris_generator.RegionCertificationProblem at 0x7fa0ed78f310>}

In [12]:
do_linesearch_cert = True
if do_linesearch_cert:
    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

Starting Region 1/6
min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 3.317209025457233
Smallest difference: 0.14689258966773977
Fixed eps is not success

min_eps = [0.8660254  1.65860451 0.10832327 0.8660254  0.07344629 1.62372753]
cur_eps = [1.29903811 2.48790677 0.16248491 1.29903811 0.11016944 2.4355913 ]
max_eps = [1.73205081 3.31720903 0.21664655 1.73205081 0.14689259 3.24745507]
Biggest difference: 1.6586045127286164
Smallest difference: 0.07344629483386989
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.62634388 1.24395338 0.08124245 0.46510824 0.05508472 1.21779565]
max_eps = [1.25268776 2.48790677 0.1624849  0.93021648 0.11016943 2.4355913 ]
Biggest difference: 2.487906768142779
Smallest difference: 0.11016943388782734
Fixed eps is not success

min_eps = [0.62634388 1.24395338 0.0812

Fixed eps is not success

min_eps = [0.99074381 2.4347021  0.15900987 0.25879608 0.10781327 2.38350541]
cur_eps = [0.99860686 2.45402513 0.16027185 0.26085001 0.10866893 2.40242212]
max_eps = [1.0064699  2.47334816 0.16153383 0.26290395 0.10952459 2.42133883]
Biggest difference: 0.0386460650410374
Smallest difference: 0.001711321708748284
Fixed eps is not success

min_eps = [0.99860686 2.45402513 0.16027185 0.26085001 0.10866893 2.40242212]
cur_eps = [1.00253838 2.46368665 0.16090284 0.26187698 0.10909676 2.41188047]
max_eps = [1.0064699  2.47334816 0.16153383 0.26290395 0.10952459 2.42133883]
Biggest difference: 0.0193230325205187
Smallest difference: 0.000855660854374149
Fixed eps is not success

min_eps = [1.00253838 2.46368665 0.16090284 0.26187698 0.10909676 2.41188047]
cur_eps = [1.00450414 2.4685174  0.16121834 0.26239047 0.10931067 2.41660965]
max_eps = [1.0064699  2.47334816 0.16153383 0.26290395 0.10952459 2.42133883]
Biggest difference: 0.00966151626025935
Smallest differenc

Fixed eps is not success

min_eps = [1.74436424 2.46914418 1.13060524 0.73288949 0.13371111 1.35528674]
cur_eps = [1.86896169 2.64551162 1.21136275 0.78523874 0.1432619  1.45209294]
max_eps = [1.99355913 2.82187906 1.29212027 0.83758799 0.1528127  1.54889913]
Biggest difference: 0.35273488293695543
Smallest difference: 0.019101587317838287
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.93448084 1.28740559 0.60568138 0.31621961 0.07163095 0.59687776]
max_eps = [1.86896169 2.57481119 1.21136275 0.63243922 0.1432619  1.19375551]
Biggest difference: 2.57481118686663
Smallest difference: 0.14326190491649007
Fixed eps is not success

min_eps = [0.93448084 1.28740559 0.60568138 0.31621961 0.07163095 0.59687776]
cur_eps = [1.40172126 1.93110839 0.90852207 0.47432941 0.10744643 0.89531664]
max_eps = [1.86896169 2.57481119 1.21136275 0.63243922 0.1432619  1.19375551]
Biggest difference: 1.287405593433315
Smallest difference: 0.07163095245824504
Fixed eps 

Fixed eps is not success

min_eps = [1.74071237 2.20547601 0.75532653 0.58904083 0.13343116 0.60508327]
cur_eps = [1.75452754 2.22297979 0.76132118 0.59371576 0.13449014 0.60988551]
max_eps = [1.76834272 2.24048357 0.76731584 0.59839069 0.13554912 0.61468776]
Biggest difference: 0.03500755572381431
Smallest difference: 0.0021179549340501758
Fixed eps is not success

min_eps = [1.75452754 2.22297979 0.76132118 0.59371576 0.13449014 0.60988551]
cur_eps = [1.76143513 2.23173168 0.76431851 0.59605322 0.13501963 0.61228664]
max_eps = [1.76834272 2.24048357 0.76731584 0.59839069 0.13554912 0.61468776]
Biggest difference: 0.017503777861906933
Smallest difference: 0.001058977467025074
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.88071757 1.11213682 0.29017409 0.29802661 0.06750981 0.20254536]
max_eps = [1.76143513 2.22427365 0.58034819 0.59605322 0.13501962 0.40509073]
Biggest difference: 2.2242736477493406
Smallest difference: 0.13501962048757432
Fix

Fixed eps is not success

min_eps = [1.31914181 1.62105459 0.14944714 0.43935253 0.10111432 0.07613411]
cur_eps = [1.53899877 1.89123036 0.17435499 0.51257796 0.11796671 0.08882313]
max_eps = [1.75885574 2.16140612 0.19926285 0.58580338 0.13481909 0.10151215]
Biggest difference: 0.5403515311722531
Smallest difference: 0.025378037854201965
Fixed eps is not success

min_eps = [1.53899877 1.89123036 0.17435499 0.51257796 0.11796671 0.08882313]
cur_eps = [1.64892726 2.02631824 0.18680892 0.54919067 0.1263929  0.09516764]
max_eps = [1.75885574 2.16140612 0.19926285 0.58580338 0.13481909 0.10151215]
Biggest difference: 0.27017576558612655
Smallest difference: 0.01268901892710099
Fixed eps is not success

min_eps = [1.64892726 2.02631824 0.18680892 0.54919067 0.1263929  0.09516764]
cur_eps = [1.7038915  2.09386218 0.19303589 0.56749702 0.130606   0.0983399 ]
max_eps = [1.75885574 2.16140612 0.19926285 0.58580338 0.13481909 0.10151215]
Biggest difference: 0.1350878827930635
Smallest difference

Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0.]
cur_eps = [0.87921318 1.04050937 0.02481897 0.26168188 0.04582369 0.05072289]
max_eps = [1.75842636 2.08101875 0.04963793 0.52336377 0.09164738 0.10144579]
Biggest difference: 2.081018749667125
Smallest difference: 0.04963793414594312
Fixed eps is not success

min_eps = [0.87921318 1.04050937 0.02481897 0.26168188 0.04582369 0.05072289]
cur_eps = [1.31881977 1.56076406 0.03722845 0.39252282 0.06873553 0.07608434]
max_eps = [1.75842636 2.08101875 0.04963793 0.52336377 0.09164738 0.10144579]
Biggest difference: 1.0405093748335625
Smallest difference: 0.02481896707297156
Fixed eps is not success

min_eps = [1.31881977 1.56076406 0.03722845 0.39252282 0.06873553 0.07608434]
cur_eps = [1.53862306 1.82089141 0.04343319 0.4579433  0.08019145 0.08876506]
max_eps = [1.75842636 2.08101875 0.04963793 0.52336377 0.09164738 0.10144579]
Biggest difference: 0.5202546874167813
Smallest difference: 0.012409483536485778
Fixed eps

Fixed eps is not success

Completed Region 2/6 in 130.87746214866638s

Starting Region 3/6
min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.79045779 1.07742201 0.73835444 0.94159301 0.65462879 0.99369636
 0.09409693 0.02882031 0.02846672 0.03253072 0.01764522]
max_eps = [1.58091559 2.15484403 1.47670889 1.88318603 1.30925759 1.98739273
 0.18819386 0.05764061 0.05693344 0.06506144 0.03529044]
Biggest difference: 2.154844026300648
Smallest difference: 0.035290436440415185
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.32819294 0.538711   0.36917719 0.4707965  0.32731434 0.49684818
 0.04704835 0.01441009 0.01423337 0.01626537 0.00882262]
max_eps = [0.65638589 1.077422   0.73835438 0.941593   0.65462868 0.99369635
 0.09409669 0.02882018 0.02846673 0.03253074 0.01764524]
Biggest difference: 1.0774219984375943
Smallest difference: 0.01764524181987929
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0

Fixed eps is success

min_eps = [0.02633074 0.03861384 0.03628332 0.03177951 0.01371899 0.02621957
 0.0036186  0.00105426 0.00144345 0.00104642 0.00145994 0.0009433
 0.00077811 0.00141212 0.0013562 ]
cur_eps = [0.02698901 0.03957919 0.0371904  0.032574   0.01406196 0.02687506
 0.00370907 0.00108062 0.00147954 0.00107258 0.00149643 0.00096688
 0.00079757 0.00144743 0.00139011]
max_eps = [0.02764727 0.04054454 0.03809748 0.03336849 0.01440494 0.02753055
 0.00379953 0.00110697 0.00151562 0.00109874 0.00153293 0.00099047
 0.00081702 0.00148273 0.00142401]
Biggest difference: 0.0019306921707367766
Smallest difference: 3.890567283732242e-05
Fixed eps is success

min_eps = [0.02633074 0.03861384 0.03628332 0.03177951 0.01371899 0.02621957
 0.0036186  0.00105426 0.00144345 0.00104642 0.00145994 0.0009433
 0.00077811 0.00141212 0.0013562 ]
cur_eps = [0.02665987 0.03909652 0.03673686 0.03217676 0.01389048 0.02654731
 0.00366384 0.00106744 0.00146149 0.0010595  0.00147819 0.00095509
 0.00078784 0

Fixed eps is not success

min_eps = [0.64525228 1.10510232 0.82389388 0.73587896 0.07640635 0.57319382
 0.02624334 0.09619901]
cur_eps = [0.69134173 1.1840382  0.88274345 0.78844174 0.08186395 0.61413624
 0.02811787 0.10307037]
max_eps = [0.73743118 1.26297408 0.94159301 0.84100453 0.08732154 0.65507865
 0.02999239 0.10994173]
Biggest difference: 0.15787175997702918
Smallest difference: 0.0037490490918443697
Fixed eps is success
Fixed mults is success

min_eps = [0. 0. 0. 0. 0. 0. 0. 0.]
cur_eps = [0.27284993 0.31906641 0.44137162 0.3821781  0.04093197 0.23345371
 0.01405893 0.05153519]
max_eps = [0.54569985 0.63813283 0.88274324 0.76435621 0.08186395 0.46690741
 0.02811787 0.10307037]
Biggest difference: 0.8827432378972678
Smallest difference: 0.028117868217515555
Fixed eps is not success

min_eps = [0.27284993 0.31906641 0.44137162 0.3821781  0.04093197 0.23345371
 0.01405893 0.05153519]
cur_eps = [0.40927489 0.47859962 0.66205743 0.57326716 0.06139796 0.35018056
 0.0210884  0.077302

Fixed eps is not success

min_eps = [0.00576675 0.00551282 0.39625672 0.32261802 0.0388808  0.00631909
 0.01335442 0.04895267]
cur_eps = [0.00865012 0.00826923 0.59438509 0.48392703 0.0583212  0.00947864
 0.02003163 0.073429  ]
max_eps = [0.0115335  0.01102563 0.79251345 0.64523604 0.0777616  0.01263819
 0.02670884 0.09790534]
Biggest difference: 0.3962567238198538
Smallest difference: 0.005512816952978905
Fixed eps is not success

min_eps = [0.00865012 0.00826923 0.59438509 0.48392703 0.0583212  0.00947864
 0.02003163 0.073429  ]
cur_eps = [0.01009181 0.00964743 0.69344927 0.56458154 0.0680414  0.01105842
 0.02337023 0.08566717]
max_eps = [0.0115335  0.01102563 0.79251345 0.64523604 0.0777616  0.01263819
 0.02670884 0.09790534]
Biggest difference: 0.1981283619099269
Smallest difference: 0.002756408476489452
Fixed eps is not success

min_eps = [0.01009181 0.00964743 0.69344927 0.56458154 0.0680414  0.01105842
 0.02337023 0.08566717]
cur_eps = [0.01081265 0.01033653 0.74298136 0.6049087

In [13]:
# uncomment to visualize planes (TODO)
# visualizer.region_to_collision_pair_to_plane_dictionary = iris_generator.linesearch_regions_to_certificates_by_collision_pair_map

In [14]:
do_alt_cert = False
if do_alt_cert:
    iris_generator.certify_and_adjust_regions_by_bilinear_alternation(max_iter = 1000, 
                                                                 stopped_improving_count_threshold = 5,
                                                                 not_improving_threshold = 1e-4)

In [15]:
# plot new regions
if do_viz:
    if do_alt_cert:
        visualizer.plot_regions(iris_generator.alternation_search_regions, region_suffix = '_new_by_alternation')
    if do_linesearch_cert:
        visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')
    

In [16]:
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
# Solve path planning
start_time = time.time()
# trajectory through linesearch_regions
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(trajectory_start.squeeze(), trajectory_end.squeeze(), 
                                                iris_generator.linesearch_regions)
spp.set_max_velocity(0.3*np.ones(plant.num_positions()))
spp.set_extra_control_points_per_region(3)
traj_iris = spp.SolveVerbose()
visualizer.draw_traj_tspace(traj_iris, 100, 'iris')

ImportError: cannot import name 'BsplineTrajectoryThroughUnionOfHPolyhedra' from 'pydrake.all' (/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/sos_iris_certifier/sos_iris_example.runfiles/drake/bindings/pydrake/all.py)

In [ ]:
print(time.time() - start_time)
print(traj_iris.start_time())
print(traj_iris.end_time())
# draw_traj_tspace(traj_iris, 100, 'iris')
#animate Iris + spp sol
substeps = 1000
its = 10
visualizer.animate_t(traj_iris, substeps, its*substeps)